This notebook obtains voting data from the Chilean Chamber of Deputies. This data were analyzed in the paper "Parliamentary roll-call voting as a complex dynamical system: The case of Chile" by Diego Morales-Bader, Ramón D. Castillo, Ralf F. A. Cox, and Carlos Ascencio-Garrido.

In [ ]:
# Libraries to get the data from the web http://opendata.camara.cl
from urllib.request import urlopen
from xml.dom import minidom

# To store data in a dataframe
import pandas as pd

# To format voting dates
import datetime

In [ ]:
# Lists of fields to be downloaded
Id_List = []
Fecha_List = []
Desc_List = []
Si_List = []
No_List = []
Abs_List = []
Disp_List = []
Resultado_List = []
Tipo_List = []
Quorum_List = []

# Dataframe
df = pd.DataFrame(columns=['ID', "Fecha", 'Descripción', "Si", "No", "Abstención", "Dispensado", "Resultado", "Tipo", "Quorum"])

# Range of years to download
from_year = 2002
to_year = 2021

# iterate over years
for i in range(from_year, to_year + 1):
    # Link to the data
    url_str = "http://opendata.camara.cl/camaradiputados/WServices/WSLegislativo.asmx/retornarVotacionesXAnno?prmAnno=" + str(i)
    url = urlopen(url_str) # open the url
    mydoc = minidom.parse(url) # parse the content
    # Get the elements
    Votaciones = mydoc.getElementsByTagName("Votacion")
    for votacion in Votaciones:
        id = votacion.getElementsByTagName("Id")[0]
        fecha = votacion.getElementsByTagName("Fecha")[0]
        desc = votacion.getElementsByTagName("Descripcion")[0]
        si = votacion.getElementsByTagName("TotalSi")[0]
        no = votacion.getElementsByTagName("TotalNo")[0]
        abs = votacion.getElementsByTagName("TotalAbstencion")[0]
        disp = votacion.getElementsByTagName("TotalDispensado")[0]

        try:
            resultado = votacion.getElementsByTagName("Resultado")[0]
            Resultado_List.append(str(resultado.firstChild.nodeValue))
        except:
            Resultado_List.append("")

        tipo = votacion.getElementsByTagName("Tipo")[0]
        quorum = votacion.getElementsByTagName("Quorum")[0]

        # Store the elements in the lists
        Id_List.append(str(id.firstChild.data))
        Fecha_List.append(str(fecha.firstChild.data))
        Desc_List.append(str(desc.firstChild.data))
        Si_List.append(str(si.firstChild.data))
        No_List.append(str(no.firstChild.data))
        Abs_List.append(str(abs.firstChild.data))
        Disp_List.append(str(disp.firstChild.data))
        Tipo_List.append(str(tipo.firstChild.nodeValue))
        Quorum_List.append(str(quorum.firstChild.nodeValue))

    print('Year:', i)

# Asiggn each list to the dataframe
df["ID"] = Id_List
df["Fecha"] = Fecha_List
df["Descripción"] = Desc_List
df["Si"] = Si_List
df["No"] = No_List
df["Abstención"] = Abs_List
df["Dispensado"] = Disp_List
df["Resultado"] = Resultado_List
df["Tipo"] = Tipo_List
df["Quorum"] = Quorum_List

In [ ]:
# Convert string to int
df = df.astype({"Si":"int", "No":"int", "Abstención":"int", "Dispensado":"int"})
# Convert the date column ('Fecha') to datetime format
df['Fecha'] = pd.to_datetime(df['Fecha'])
df = df.sort_values(by='Fecha', ignore_index=True)

In [ ]:
df.head()

In [ ]:
# Calculate missing voting outcomes
def missing_voting_outcome(row):
    total_session = row['Si'] + row['No'] + row['Abstención']
    if total_session > 0:
        percentage_yes_simple = row['Si'] / total_session* 100
    else:
        percentage_yes_simple = total_session
    if (row["Resultado"] == "") and (row["Quorum"] == 'Quórum Simple'):
        if percentage_yes_simple > 50:
            return 'Aprobado'
        elif percentage_yes_simple <= 50:
            return "Unánime"
    elif row["Resultado"] == "" and row["Quorum"] == 'Quórum Calificado':
        if row['Si'] / 120 - row['Dispensado'] > 0.5:
            return 'Aprobado'
        elif row['Si'] / 120 - row['Dispensado'] <= 0.5:
            return "Unánime"
    elif row["Resultado"] == "" and row["Quorum"] == 'Ley Orgánica Constitucional':
        if row['Si'] / 120 - row['Dispensado'] >= 4/7:
            return 'Aprobado'
        if row['Si'] / 120 - row['Dispensado'] < 4/7:
            return "Unánime"
    elif row["Resultado"] == "" and (row["Quorum"] == 'Reforma Constitucional 2/3' or row["Quorum"] == '2/3'):
        if row['Si'] / 120 - row['Dispensado'] >= 2/3:
            return 'Aprobado'
        if row['Si'] / 120 - row['Dispensado'] < 2/3:
            return 'Unánime'
    elif row["Resultado"] == "" and (row["Quorum"] == 'Reforma Constitucional 3/5' or row["Quorum"] == '3/5'):
        if row['Si'] / 120 - row['Dispensado'] >= 3/5:
            return 'Aprobado'
        if row['Si'] / 120 - row['Dispensado'] < 3/5:
            return 'Unánime'
    elif row["Resultado"] == "" and row["Quorum"] == '1/3':
        if row['Si'] / 120 - row['Dispensado'] >= 1/3:
            return 'Aprobado'
        if row['Si'] / 120 - row['Dispensado'] < 1/3:
            return 'Unánime'
    elif row["Resultado"] != "":
        return row["Resultado"]

df['Resultados2'] = df.apply(lambda row: missing_voting_outcome(row), axis=1)

For votes of the type "Proyecto de Acuerdo" and "Proyecto de Resolución" that have as a voting outcome "No Quorum", they are assigned the outcome of "rejected" according to the indications of article 114 of the rules of procedure of the Chamber of Deputies of Chile.

In [ ]:
def no_quorum_AC_R(row):
    if (row["Tipo"] == "Proyecto de Acuerdo") or (row["Tipo"] == "Proyecto de Resolución"):
        if row["Resultados2"] == "Sin Quórum":
            return 'Unánime'
        else:
            return row["Resultados2"]
    else:
        return row["Resultados2"]

df['Resultados2'] = df.apply(lambda row: no_quorum_AC_R(row), axis=1)

Recodes ties as rejections

Voting outcomes are categorized as follows:
- Pass or Approved votes = 1
- Rejected votes = 2
- Ties are considered as rejected votes.
- Votes without quorum are considered rejected.

In [ ]:
# Ties are considered as rejected votes
def ties(row):
    if row['Resultados2'] == 'Empate':
        return 'Unánime'
    else:
        return row["Resultados2"]

# Categorize voting outcome (df["Resultados2"])
def voting_outcome_cat(row):  
    if row["Resultados2"] == "Aprobado":
        return 1
    elif row["Resultados2"] == "Unánime":
        return 2
    elif row["Resultados2"] == "Sin Quórum":
        return 2

df['Resultados2'] = df.apply(lambda row: ties(row), axis=1)
df['voting_outcome'] = df.apply(lambda row: voting_outcome_cat(row), axis=1)

The following changes are made due to discrepancies between session logs and downloaded data. Corrections are made according to the session logs.

In [ ]:
# Function to replace the voting outcome
def replace_outcome(ID, resultado, outcome):
    index = df[df.ID==ID].index[0]
    df.loc[index, 'Resultados2'] = resultado
    df.loc[index, 'voting_outcome'] = outcome

# Function to remove votes from the data
def delete_vote(ID):
    if df[df.ID==ID].index.size != 0:
        index = df[df.ID==ID].index[0]
        df.drop(int(index), inplace = True)

# Correction of some voting records (votes indicated by their ID)
replace_outcome("3634","Unánime","2")
replace_outcome("3769","Unánime","2")
replace_outcome("14563","Unánime","2")
replace_outcome("16121","Unánime","2")
replace_outcome("16227","Unánime","2")
replace_outcome("16253","Unánime","2")
replace_outcome("16259","Unánime","2")
replace_outcome("34258","Unánime","2")
replace_outcome("34760","Unánime","2")
replace_outcome("14527","Aprobado","1")
replace_outcome("15831","Aprobado","1")
replace_outcome("16360","Aprobado","1")
replace_outcome("18242","Aprobado","1")
replace_outcome("10896","Aprobado","1")
replace_outcome("10897","Aprobado","1")
replace_outcome("15276","Unánime","2")
replace_outcome("15277","Unánime","2")

# Votes that are eliminated due to recording errors and not being found in the session logs.
delete_vote("30166")
delete_vote("31032")
delete_vote("30997")
delete_vote("10264")
delete_vote("32387")
delete_vote("2342")
delete_vote("32387")
delete_vote("2672")
delete_vote("2673")
delete_vote("36978")

# Changes in the type of quorum required for passing the ballot
index = df[df.ID=="12411"].index[0]
df.loc[index, 'Quorum'] = "Ley Orgánica Constitucional"

index = df[df.ID=="16662"].index[0]
df.loc[index, 'Quorum'] = "3/5"

index = df[df.ID=="18545"].index[0]
df.loc[index, 'Quorum'] = "Quórum Calificado"

index = df[df.ID=="32860"].index[0]
df.loc[index, 'Quorum'] = "Ley Orgánica Constitucional"

# Changes in the number of votes and the voting result for ballots that were listed with zero votes
index = df[df.ID=="2379"].index[0]
df.loc[index, 'Si'] = 111

index = df[df.ID=="34343"].index[0]
df.loc[index, 'Si'] = 149
replace_outcome("34343","Aprobado","1")

index = df[df.ID=="34344"].index[0]
df.loc[index, 'Si'] = 149
replace_outcome("34344","Aprobado","1")

index = df[df.ID=="34345"].index[0]
df.loc[index, 'Si'] = 149
replace_outcome("34345","Aprobado","1")

index = df[df.ID=="34346"].index[0]
df.loc[index, 'Si'] = 149
replace_outcome("34346","Aprobado","1")

index = df[df.ID=="34448"].index[0]
df.loc[index, 'Si'] = 142
replace_outcome("34448","Aprobado","1")

index = df[df.ID=="34449"].index[0]
df.loc[index, 'Si'] = 142
replace_outcome("34449","Aprobado","1")

index = df[df.ID=="34450"].index[0]
df.loc[index, 'Si'] = 142
replace_outcome("34450","Aprobado","1")

index = df[df.ID=="10008"].index[0]
df.loc[index, 'Si'] = 97
replace_outcome("10008","Aprobado","1")

index = df[df.ID=="10311"].index[0]
df.loc[index, 'Si'] = 79
replace_outcome("10311","Aprobado","1")

index = df[df.ID=="34723"].index[0]
df.loc[index, 'Si'] = df.loc[index-1, 'Si'] + df.loc[index-1, 'No'] + df.loc[index-1, 'Abstención']
replace_outcome("34723","Aprobado","1")

index = df[df.ID=="34838"].index[0]
df.loc[index, 'Si'] = df.loc[index-1, 'Si'] + df.loc[index-1, 'No'] + df.loc[index-1, 'Abstención']
replace_outcome("34838","Aprobado","1")

index = df[df.ID=="34920"].index[0]
df.loc[index, 'Si'] = 93
replace_outcome("34920","Aprobado","1")

index = df[df.ID=="10899"].index[0]
df.loc[index, 'Si'] = 33
df.loc[index, 'Abstención'] = 1
replace_outcome("10899","Unánime","2")

index = df[df.ID=="15290"].index[0]
df.loc[index, 'Si'] = 105
replace_outcome("15290","Aprobado","1")

Deletes all votes that do not meet the minimum quorum. These votes are repeated until the quorum is met.

In [ ]:
print('Number of votes to delete:', len(df.index[df['Resultados2'] == "Sin Quórum"]))
# Elimination of votes catalogued as "no quorum"
df.drop(df.index[df['Resultados2'] == "Sin Quórum"], inplace=True)
# Reset index
df.reset_index(drop=True, inplace=True)

Compute the Agreement Ratio: Number of votes of the majority option divided by the total number of votes (including abstentions)

In [ ]:
# Compute the Agreement Ratio
def agreement_ratio(row):  
    if row['Si'] > row['No']:
        total = row['Si'] + row['No'] + row['Abstención']
        if total > 0:
            ar = row['Si'] / total
            return ar
    elif row['Si'] <= row['No']:
        total = (row['Si'] + row['No'] + row['Abstención'])
        if total > 0:
            ar = row['No'] / (row['Si'] + row['No'] + row['Abstención'])
            return ar

df['agreement_ratio'] = df.apply(lambda row: agreement_ratio(row), axis=1)

In [ ]:
# Asign the legislative session
def legislative_term(row):  
    if datetime.datetime(2002,3,11) <= row["Fecha"] < datetime.datetime(2006,3,11):
        return "2002-2006"
    elif datetime.datetime(2006,3,11) <= row["Fecha"] < datetime.datetime(2010,3,11):
        return "2006-2010"
    elif datetime.datetime(2010,3,11) <= row["Fecha"] < datetime.datetime(2014,3,11):
        return "2010-2014"
    elif datetime.datetime(2014,3,11) <= row["Fecha"] < datetime.datetime(2018,3,11):
        return "2014-2018"
    elif datetime.datetime(2018,3,11) <= row["Fecha"] < datetime.datetime(2022,3,11):
        return "2018-2022"
    elif datetime.datetime(2022,3,11) <= row["Fecha"] < datetime.datetime(2026,3,11):
        return "2018-2022"

df['legislative_term'] = df.apply(lambda row: legislative_term(row), axis=1)

Cleaning the dataframe and renaming the columns

In [ ]:
df.drop(['Resultado'], axis=1, inplace=True)
df.rename(columns = {'Fecha':'Date', 'Descripción':'Description', 'Si':'Yes',
                    'Abstención':'Abstention', 'Dispensado':'Dispensed',
                    'Tipo':'Type', 'Resultados2':'voting_outcome_name'}, inplace = True)

df['Type'].replace(['Proyecto de Ley', 'Proyecto de Acuerdo', 'Proyecto de Resolución', 'Otros'],
                   ['Law Project', 'Agreement Project', 'Resolution Project', 'Other'], inplace=True)
df['voting_outcome_name'].replace(['Aprobado', 'Unánime'],
                                  ['Approved', 'Rejected'], inplace=True)


| Column              | Meaning                                                                                                                    |
|---------------------|----------------------------------------------------------------------------------------------------------------------------|
| ID                  | Identifier code of each vote                                                                                               |
| Date                | Date of each vote (yyyy-mm-dd hh:mm:ss)                                                                                    |
| Description         | Brief description of the content of the vote. In most cases it only indicates the bulletin number to which it corresponds. |
| Yes                 | Number of votes in favor                                                                                                   |
| No                  | Number of votes against                                                                                                    |
| Abstention          | Number of deputies who abstained from voting                                                                               |
| Dispensed           | Number of deputies excused to be absent for each vote                                                                      |
| Type                | Type of vote, which can be: Bill (or Law Project), Agreement Project, Resolution Project, or Other.                        |
| Quorum              | Type of quorum required for a vote to pass                                                                                 |
| voting_outcome_name | Name of voting outcome: Approved (Pass) or Reject                                                                          |
| voting_outcome      | Numerical coding of voting outcome: Approved (Pass) = 1, Rejected = 2                                                      |
| agreement_ratio     | Number of votes of the majority option divided by the total number of votes (including abstentions)                        |
| legislative_term    | Legislative term in which each vote occurs                                                                                 |

Agreement Project: It is the proposal that 1 and up to 10, deputies and/or deputies present in writing to the Chamber, with the purpose of exercising the powers conferred by Article 52, number 1, first paragraph of letter a), of the Political Constitution, to adopt agreements or suggest observations on the acts of the Government, having to so express it, invoking in its text said constitutional norm.

Resolution Project: It is the proposal that 1 and up to 10, deputies and/or deputies present in writing to the Chamber with the purpose of obtaining a pronouncement of the Chamber of Deputies and Deputies on issues of general interest, both national and international, expressing their concern about them.

Other: Constitutional accusations, votes to establish investigatory commissions, and other internal matters of the chamber that do not constitute a law or bill.

In [ ]:
df.head()

Saves the data used for the study in csv format. The agreement_ratio and voting_outcome columns were considered for the analyses.

In [ ]:
# Store the data
df.to_csv(f"votes_{from_year}-{to_year}.csv", encoding="utf-8")

Those who are interested can remove more votes that in theory should not meet the minimum quorum. A simple rule is used, which eliminates all ballots whose sum of votes is equal to or less than 38 votes. However, this rule may eliminate valid votes. It would be necessary to review possible changes in legislation of the Chamber of Deputies and minimum quorums to be sure that this rule is applicable.

In [ ]:
# Deletes all votes that do not meet the minimum quorum and are not registered as such
# Computes the sum of the votes
df['sum'] = df['Yes'] + df['No'] + df['Abstention']
# Votes where 38 or fewer deputies voted do not have the minimum quorum required.
df = df[df['sum'] > 38]
# Drop sum column
df.drop(['sum'], axis=1, inplace=True)
# Reset index
df.reset_index(drop=True, inplace=True)

# Store the data
df.to_csv(f"votes_{from_year}-{to_year}_v2.csv", encoding="utf-8")